In [1]:
import os
import numpy as np
import tqdm
import json
import random
import time
import multiprocessing
from ember_features import PEFeatureExtractor
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
import copy
import matplotlib.pyplot as plt
import scipy as stats
from sklearn.manifold import TSNE
from collections import defaultdict

plt.rcParams['font.size'] = 18
#plt.rcParams['font.family'] = "serif"
tdir = 'in'
major = 5.0
minor = 3.0
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor

np.random.RandomState(42);



### planned graphs

- test

In [ ]:
all_task_months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                   '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']

malware_family_sample_count = {}
goodware_family_sample_count = {}

malware_family_samples = {}

others_family = {}

for task in range(0,len(all_task_months)):
    
    current_task = all_task_months[task]
    task_months = all_task_months[:task+1]
    
    
    save_dir = '/home/mr6564/continual_research/month_based_processing_with_family_labels/' + str(current_task) + '/'
    
    print(f'Processing data for task {current_task}')
    Y_family_labels_file = save_dir + 'task_family_labels.npz'
    Y_fam_labels_ = np.load(Y_family_labels_file)
    Y_fam_labels = Y_fam_labels_['family_labels']

    y_path = os.path.join(save_dir, "y_train.dat")
    X_path = os.path.join(save_dir, "X_train.dat")
    
    
    y_ = np.memmap(y_path, dtype=np.float32, mode="r")
    N = y_.shape[0]
    
    ndim = 2381
    X_ = np.memmap(X_path, dtype=np.float32, mode="r", shape=(N, ndim))    
    #print(np.unique(y_))
    
    Y_family_labels_file = save_dir + 'task_family_labels.npz'
    Y_fam_labels_ = np.load(Y_family_labels_file)
    Y_fam_labels = Y_fam_labels_['family_labels']
    
    
    goodware_indices = []
    malware_indices = []
    
    others_family_samples = []
    
    malware_task_family_samples = defaultdict(list)
    
    for ind, i in enumerate(y_):
        if i == 0:
            goodware_indices.append(ind)
        elif i == 1:
            malware_indices.append(ind)
            if Y_fam_labels[ind] == '':
                others_family_samples.append(X_[ind])
            else:
                malware_task_family_samples[Y_fam_labels[ind]].append(X_[ind])
                #if Y_fam_labels[ind] not in malware_task_family_samples.keys():
                #    malware_task_family_samples[Y_fam_labels[ind]]= X_ind
                #else:
                #    malware_task_family_samples[Y_fam_labels[ind]].append(X_ind)
        else:
            pass
    
    malware_family_samples[task] = malware_task_family_samples
    others_family[task] = others_family_samples
    

    Y_families_malware = Y_fam_labels[malware_indices]
    Y_families_goodware = Y_fam_labels[goodware_indices]
    
    
    
    for yfam in Y_families_malware:
        if yfam in malware_family_sample_count.keys():
            malware_family_sample_count[yfam] += 1
        else:
            malware_family_sample_count[yfam] = 1
    
    
    for yfam in Y_families_goodware:
        if yfam in goodware_family_sample_count.keys():
            goodware_family_sample_count[yfam] += 1
        else:
            goodware_family_sample_count[yfam] = 1
            
            
    

print(len(malware_family_sample_count.keys()), len(goodware_family_sample_count.keys()))